## Павел Клеменков
## Chief Data Scientist @ NVIDIA
## Founder @ Moscow Spark (Telegram @moscowspark)

In [ ]:
from IPython.display import IFrame, Image

## Мотивация создания Apache Spark

### Рассмотрим два примера приложений:
- Обучить модель на больших данных (читай итеративный алгоритм над фиксированным датасетом)
- Провести ad-hoc анализ данных из двух таблиц (читай несколько интерактивных запросов с джойнами)

## Основные недостатки классического MapReduce
- Быстроумирающие контейнеры
- Постоянное чтение/запись во внешнее хранилище
- Сложный API
- Ограниченное число источников/приемников данных
- MapReduce - это только вычислительный фреймворк

## Apache Spark - это *быстрая* распределенная вычислительная платформа *общего назначения*
1. **Быстрая** - это в памяти и с ленивыми вычислениями
2. **Общего назначения** - значит на ней можно реализовать любые вычисления (батчевые, интерактивные, итеративные, в режиме реального времени)

<img src="pics/spark_stack.png" width=1000/>

## Множество источников данных

<img src="pics/spark_data_sources.jpg" width=1000/>

## Архитектура Apache Spark

<img src="pics/cluster-overview.png" width=800/>

## Запуск PySpark

In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

## Другой способ запуска
```bash
% export PYSPARK_PYTHON=python3
% export PYSPARK_DRIVER_PYTHON=jupyter
% export PYSPARK_DRIVER_PYTHON_OPTIONS='notebook --ip="*" --no-browser'
% pyspark
```

## SparkContext (sc) - это основной управляющий объект.

In [ ]:
sc

## Для получения всех установленных опций конфигурации можно использовать `sc.getConf()`

In [ ]:
sc.getConf().getAll()

## Существует два способа создать RDD
- распределить коллекцию объектов с драйвера
- загрузить внешний датасет

## 1. Распределить коллекцию с драйвера

In [ ]:
import numpy as np
vocabulary = ("Apache", "Spark", "Hadoop")
numbers = np.random.randint(10, size=10000)
words = np.random.choice(vocabulary, size=10000)
collection = zip(numbers, words)

In [ ]:
rdd = sc.parallelize(collection)

In [ ]:
rdd

In [ ]:
rdd.count()

In [ ]:
rdd.take(10)

## 2. Загрузить внешний датасет (датасет загружается из HDFS)

In [ ]:
!hdfs dfs -ls /user/pavel.klemenkov/lectures/lecture01/data/ips.txt

In [ ]:
rdd2 = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/ips.txt")

In [ ]:
rdd2.take(10)

In [ ]:
rdd2.count()

## RDD API состоит из операции двух типов:
- action
- transformation

### Трансформация преобразовывает RDD в другой RDD и не приводит к вычислению графа

In [ ]:
rdd = sc.parallelize(range(100))

In [ ]:
rdd

### Action заставляет Spark вычислить граф и вернуть результат либо на драйвер, либо во внешнее хранилище

In [ ]:
rdd.count()

In [ ]:
rdd.take(10)

### Трансформации можно применять одну за другой, никаких вычислений не будет сделано, пока не будет вызван action

In [ ]:
rdd

In [ ]:
rdd2 = rdd.filter(lambda x: x % 2)
rdd2

In [ ]:
rdd3 = rdd2.map(lambda x: x * 2)
rdd3

In [ ]:
rdd3.collect()

### `take()` пытается минимизировать число обращений к партициям, поэтому может возвращать смещенные результаты

In [ ]:
rdd.take(10)

### Будьте аккуратны с `collect()`, потому что он загружает все данные из RDD на драйвер. Это может легко привести к Out of Memory exception

In [ ]:
rdd.collect()[:20]

### Если нужно получить небольшое число записей на драйвер и, при этом, сохранить распределение, то лучше сделать выборку

In [ ]:
rdd.takeSample(withReplacement=False, num=20, seed=5757)

## Познакомимся с данными. Будем работать с двумя таблицами

![](pics/data_table1.png)

![](pics/data_table2.png)

### Примеры трансформаций

In [ ]:
rdd = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/ips.txt")

In [ ]:
rdd.take(5)

In [ ]:
ips = rdd.map(lambda x: x.split("\t"))

In [ ]:
ips.take(5)

In [ ]:
ips_filtered = ips.filter(lambda x: x[1] != "CHINA")

In [ ]:
ips_filtered.take(5)

In [ ]:
raw_logs = sc.textFile("/user/pavel.klemenkov/lectures/lecture01/data/log.txt")

In [ ]:
raw_logs.take(5)

In [ ]:
logs = raw_logs.map(lambda x: x.split("\t"))

In [ ]:
logs.take(5)

In [ ]:
logs.flatMap(lambda x: x[2].split()).take(20)

## Зачем нужны отдельные трансформации и отдельные action?

![](pics/dag1.png)

![](pics/dag2.png)

### Последовательность трансформаций определяет граф вычислений (DAG - direct acyclic graph). В нем есть партиции и зависимости между партициями. Таким образом Spark имеет всю необходимую информацию для вычилсения графа в любой точке и возможных оптимизаций

![](pics/dag3.png)

### Трансформации бывают *узкими*

![](pics/narrow_transformation.png)

### И *широкими*

![](pics/wide_transformation.png)

### Широкие трансформации разделяют джоб на стейджи. Между стейджами происходит shuffle данных, которого надо избегать

## Персистентность и кэширование

### RDD вычисляются лениво, когда вызывается action. Часто мы хотим вызвать несколько actions для одного и тоге же RDD. Если мы просто сделаем это, то граф будет полностью перевычисляться каждый раз.

In [ ]:
ips.count()

In [ ]:
ips.top(10)

### Чтобы этого избежать, мы можем закэшировать RDD в памяти. Кэширование произойдет при вызове первого action.

In [ ]:
ips.cache()

In [ ]:
ips.count()

In [ ]:
ips.top(20)

### `cache()` сохраняет RDD в памяти. Для большего контроля можно использовать `persist(storage_level)`:
+ MEMORY_ONLY
+ MEMORY_AND_DISK
+ DISK_ONLY
+ MEMORY_ONLY_2
+ MEMORY_AND_DISK_2

### Все сохраненные RDD можно увидеть во вкладке "Storage" Spark UI
### Или более программатичным способом

In [ ]:
from pyspark import StorageLevel

In [ ]:
StorageLevel(False, True, False, False, 1)

In [ ]:
ips.getStorageLevel()

In [ ]:
ips.unpersist()

In [ ]:
ips.persist(StorageLevel.DISK_ONLY_2)

In [ ]:
ips.getStorageLevel()

## PairRDD (ключ-значение)

### PairRDD - это RDD для работы с парами ключ-значение. Spark предполагает, что PairRDD содержить в себе объекты, состящие ровно из двух элементов! PairRDD предоставляют методы группировки, аггрегации и объединения (join) двух RDD

### Пусть есть задача подсчитать распределение кодов ERROR и WARNING в лог-файле

In [ ]:
raw_logs.take(5)

In [ ]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .collect())

In [ ]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .groupByKey()
         .map(lambda x: (x[0], len(x[1])))
         .collect())

### Или немного проще

In [ ]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))
         .countByKey()
         .items())

### Стоит заметить, что `groupByKey()` предполагает перемещение всех записей с одним ключом на один экзекьютор. В случае очень скоршенных распределений это может привести к падению экзекьютора с OOM. Поэтому всегда при группировках стоит подумать об использовании `reduceByKey()`.

In [ ]:
(raw_logs.filter(lambda x: "INFO" not in x)
         .map(lambda x: (x.split("\t")[1], 1))\
         .reduceByKey(lambda x, y: x + y)
         .collect())

## Join

### Два PairRDD можно объединить по ключу
### Поддерживаются inner join, left outer join, right outer join и full outer join

In [ ]:
logs.take(5)

In [ ]:
ips.take(5)

In [ ]:
logs.join(ips).take(5)

![](pics/Jackie-Chan-WTF.jpg)

### Не стоит забывать, что Spark предполагает, что PairRDD состоит ровно! из двух элементов, поэтому все остальные элементы просто отбрасываются!

In [ ]:
def split_logs(line):
    split = line.split("\t")
    return split[0], split[1:]

In [ ]:
logs_cached = raw_logs.map(split_logs).cache()

In [ ]:
logs_cached.take(5)

In [ ]:
logs_cached.join(ips).take(5)

## Управление параллелизмом.

### Вспомним, что атомарным уровнем параллелизма в Spark является партиция. Об этом всегда стоит помнить, когда есть проблемы с производительностью приложения

In [ ]:
logs.getNumPartitions()

### Метод `repartition()` может быть использован для изменения числа партиций.

In [ ]:
logs = logs.repartition(8)

In [ ]:
logs.getNumPartitions()

### `repartition()` всегда приводит к равномерному перераспределению данных, что ведет к shuffle. Если Вы уменьшаете число партиций, то стоит использовать `coalesce()`, который может избежать shuffle

In [ ]:
logs = logs.coalesce(10)

In [ ]:
logs.getNumPartitions()

In [ ]:
logs = logs.coalesce(4)

In [ ]:
logs.getNumPartitions()

### Узнать дефолтный уровень параллелизма можно из конфига. По-умолчанию, при работе с YARN, использукется общее число ядер, выделенных этому SparkContext на всех экзекьюторах, либо 2. Что больше.

In [ ]:
sc.getConf().get("spark.default.parallelism")

In [ ]:
sc.parallelize(range(100)).getNumPartitions()

## Broadcast

### Broadcast-объект - это неизменяемая переменная, которая разделяется между всеми экзекьюторами
### Дистрибуция broadcast-объекта производится быстро и эффективно p2p-протоколом

### Реализуем map-side join с помощью broadcast-объекта

In [ ]:
ips_local = dict(ips.collect())

In [ ]:
ips_local['192.168.0.10']

In [ ]:
ips_broadcasted = sc.broadcast(ips_local)

In [ ]:
ips_broadcasted.value['192.168.0.10']

In [ ]:
logs_cached.take(5)

In [ ]:
def resolve_ip(row):
    return ips_broadcasted.value[row[0]], row[1:] ## row[0] is the IP address

In [ ]:
logs_cached.map(resolve_ip).take(10)

## Не забудьте погасить SparkContext!

In [ ]:
sc.stop()